## Relative Strength Index & Gold Prices

The Relative Strength Index (RSI) is a popular momentum indicator in technical analysis., helping traders evaluate the strength and speed of price movements. Developed by J. Welles Wilder in the late 1970s, the RSI measures the magnitude of recent price changes to assess whether an asset is overbought or oversold. It is displayed as a line graph that moves between two extremes, 0 and 100 and is often used to identify potential reversals in market trends. The RSI can be expressed in mathematical form as:

# $RSI = 100 - \frac{100}{1 + RS}$

where...

# $RS = \frac{\text{Average of x days' up closes}}{\text{Average of x days down closes}}$

Technical Analysis has received a lot of slander online with some calling it "stock astrology". However, is this reputation deserving or is there something people are overlooking? When looking for the ticker to test it, I landed on Gold. Not because of any analytical reasoning, rather I just found that Gold was interesting to look into.

In [2]:
import yfinance as yf
import plotly.express as px

In [3]:
ticker = "GLD"
gld = yf.download(ticker, period="max", interval="1d", auto_adjust=False)

# Reset index so "Date" is a column
gld = gld.reset_index()

# Flatten MultiIndex column names if they exist
gld.columns = [col[0] if isinstance(col, tuple) else col for col in gld.columns]

print(gld.head())

[*********************100%***********************]  1 of 1 completed

        Date  Adj Close      Close       High        Low       Open    Volume
0 2004-11-18  44.380001  44.380001  44.490002  44.070000  44.430000   5992000
1 2004-11-19  44.779999  44.779999  44.919998  44.470001  44.490002  11655300
2 2004-11-22  44.950001  44.950001  44.970001  44.740002  44.750000  11996000
3 2004-11-23  44.750000  44.750000  44.919998  44.720001  44.880001   3169200
4 2004-11-24  45.049999  45.049999  45.049999  44.790001  44.930000   6105100


In [4]:
fig = px.line(gld, "Date", "Close", title = 'GOLD Price Daily Index')
fig.show()

We can see that gold has experienced some turbulence throughout its years. There are peaks and troughs. RSI seems like a candidate since it ccould potentially filter out the peaks and troughs getting a consistent return.

### What is our $n$ for average day gain/loss?

There is no rigid RSI $n$ days average to use. Most "analysts" use a 9-15 day RSI. When looking online, I found that the most common period or $n$ to use for the RSI calculation was 14 days. Hence, I came to the conclusion that 14-days would suffice.

In [5]:
def calculate_rsi(data, window=14):
    delta = data["Close"].diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0.0)
    loss = -delta.where(delta < 0, 0.0)

    # Exponential moving averages (Wilder’s smoothing)
    avg_gain = gain.ewm(alpha=1/window, min_periods=window).mean()
    avg_loss = loss.ewm(alpha=1/window, min_periods=window).mean()

    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))

    return rsi

In [6]:
gld['RSI'] = calculate_rsi(gld)
fig = px.line(gld, x="Date", y="RSI", title="RSI Strategy For Gold")
fig.add_hline(y=30, line_dash="dash", line_color="red")
fig.add_hline(y=70, line_dash="dash", line_color="green")
fig.show()

### The Intuition Behind RSI

The RSI strategy goes that whenever the RSI is greater than 70, we should sell out stock because it is overbought. Conversely, if the RSI is less than 30, we should buy the stock because the stock is oversold. Our strategy for RSI will compare a buy-and-hold strategy from 2005 up until 2025 versus a RSI strategy where we sell whenever the RSI > 80 and buy whenever RSI < 30.

In [7]:
def rsi_strategy(df, buy_level=30, sell_level=70):
    position = None   # None = no trade, "long" = bought
    buy_price = 0
    returns = []

    for i in range(len(df)):
        rsi = df["RSI"].iloc[i]
        price = df["Close"].iloc[i]

        if position is None and rsi < buy_level:
            # Buy signal
            position = "long"
            buy_price = price

        elif position == "long" and rsi > sell_level:
            # Sell signal
            ret = (price - buy_price) / buy_price
            returns.append(ret)
            position = None

    return returns

In [8]:
def rsi_strategy(df, buy_level=30, sell_level=70, initial_capital=100):
    capital = initial_capital
    shares = 0
    position = None
    equity_curve = []

    for i in range(len(df)):
        price = df["Close"].iloc[i]
        rsi = df["RSI"].iloc[i]

        # Buy: invest all capital into shares
        if position is None and rsi < buy_level:
            shares = capital / price
            capital = 0
            position = "long"

        # Sell: liquidate all shares into cash
        elif position == "long" and rsi > sell_level:
            capital = shares * price
            shares = 0
            position = None

        # Portfolio value at this step
        if position == "long":
            equity_curve.append(shares * price)
        else:
            equity_curve.append(capital)

    return equity_curve


def buy_and_hold(df, initial_capital=100):
    first_price = df["Close"].iloc[0]
    equity_curve = initial_capital * (df["Close"] / first_price)
    return equity_curve


gld["RSI Strategy"] = rsi_strategy(gld)
gld['Buy & Hold'] = buy_and_hold(gld)

### What exactly is our strategy?

A buy-and-hold strategy is pretty self-explanatory. We buy at the beginning, in this example 2005, and we sell in the end or 2025. The RSI strategy differs greatly. Since we are only going to be focusing on Gold, our RSI will be limited to Gold. In our RSI, we will buy whenever RSI is less than 30. When RSI is greater than 70, we will buy Gold. In this example, we will be keeping track of the value of our position, we will start with \$100 if we end up with more than the share price, we will buy more shares.

In [9]:

fig = px.line(
    gld,
    x="Date",
    y=["RSI Strategy", "Buy & Hold"],
    title="RSI Indicator Strategy vs Buy & Hold Strategy",
    labels={"value": "Portfolio Value ($)", "variable": "Strategy"}
)
fig.show()

### Conclusion and Discussion...

Our RSI strategy severely underperforms against a simple buy-and-hold. Our end value for the RSI strategy is ~$\$$206 compared to a buy-and-hold strategy of ~ $\$$709.

### Why is this?

RSI while a good feature to incorporate, the feature is flawed in that it misses out on compounding rates. RSI treats a stock's fair value through its past $n$. Often this means it overlooks stock rallies. RSI implicitly assumes mean reversion. RSI treats relative strength of recent gains and losses through a short window. This means that the RSI assumes that stocks have a "normal" level they return over the long term. This explains why the RSI strategy underperformed with gold, Gold has an upward trend and RSI forces the buyer to ignore big rallies as it considers the stock undervalued. That's not to discredit RSI as a fundamental strategy as RSI works great with data with a constant mean-reverting pattern. However, finding stocks with mean reversion patterns is rare as stocks typically exhibit a trend-like behavior. On top of that, trying to find stocks with a mean reversion pattern is not a great strategy as stock with long term trend will always outperform those stocks with mean reverting properties.

### Other issues with RSI

RSI requires constant rebalancing of a portfolio. continuously rebalancing means two things: taxes and fees. Most brokerages charge fees based off trades. Constantly executing trades can be expensive in the long run. Over many trades, these costs add up, eroding returns further. In fact, our experiment did not even consider trade fees. In addition to that, selling stock within a year involves paying short-term capital gains taxes. Short-term capital gains tax is a lot higher than paying long-term capital gain tax. This of course assumes you have income greater than the standard deduction.

## Conclusion:

Should you center a strategy around RSI? The short answer is no. RSI is a risky strategy with very little academic rigor behind it. RSI limits stock analysis to a short-term window that overlooks factors that could be causing stock price to exhibit a trend. As mentioned previously, there a fees associated with RSI that make it unappealing. RSI while intuitive should be treated with caution and awareness.